In [1]:
# === Librerías de modelado y análisis ===
import pandas as pd
import numpy as np

# === Preprocesamiento y modelado ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB

# === Métricas de evaluación ===
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
# === Cargar el dataset ===
path = 'D:/TFG/codigo/datos/indicadores-filtrados-TODAS-LIGAS.csv'
df = pd.read_csv(path, sep=",")

# === Eliminar columnas no predictivas ===
df.drop(['jornada', 'liga', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)

# === Definir variables predictoras e indicadores de resultado ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

y_resultado = df['resultado_partido']           # Objetivo: clasificación (1, X, 2)
y_goles_local = df['resultado_local']           # Regresión (no para Naive Bayes)
y_goles_visitante = df['resultado_visitante']   # Regresión (no para Naive Bayes)

# === Escalar los datos ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
# === División para clasificación del resultado del partido ===
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=42
)

# === División para predicción de goles del equipo local ===
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(
    X_scaled, y_goles_local, test_size=0.2, random_state=42
)

# === División para predicción de goles del equipo visitante ===
X_train_vis, X_test_vis, y_train_vis, y_test_vis = train_test_split(
    X_scaled, y_goles_visitante, test_size=0.2, random_state=42
)


### Predicciones resultado

In [7]:
# === Modelo base ===
nb_base = GaussianNB()

# === Grid de hiperparámetros ===
param_grid_nb = {
    'var_smoothing': np.logspace(-12, -6, 7)  # Valores entre 1e-12 y 1e-6
}

# === Búsqueda con validación cruzada ===
grid_search_nb = GridSearchCV(
    estimator=nb_base,
    param_grid=param_grid_nb,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Ajustar con los datos de entrenamiento
grid_search_nb.fit(X_train_cls, y_train_cls)

# Mostrar el mejor valor encontrado
print("Mejor valor de var_smoothing:", grid_search_nb.best_params_)

Mejor valor de var_smoothing: {'var_smoothing': 1e-12}


In [8]:
# === Entrenamiento final con el mejor valor de var_smoothing ===
best_nb_model = grid_search_nb.best_estimator_
best_nb_model.fit(X_train_cls, y_train_cls)

# === Predicción sobre el conjunto de prueba ===
y_pred_cls = best_nb_model.predict(X_test_cls)


In [9]:
# === Evaluación del modelo ===
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"Accuracy del modelo Naive Bayes: {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_cls, y_pred_cls))

# Informe por clase
print("Informe de clasificación:")
print(classification_report(y_test_cls, y_pred_cls, target_names=["Victoria local", "Empate", "Victoria visitante"]))


Accuracy del modelo Naive Bayes: 0.4316
Matriz de confusión:
[[138  90  49]
 [ 49  84  41]
 [ 52  64  40]]
Informe de clasificación:
                    precision    recall  f1-score   support

    Victoria local       0.58      0.50      0.53       277
            Empate       0.35      0.48      0.41       174
Victoria visitante       0.31      0.26      0.28       156

          accuracy                           0.43       607
         macro avg       0.41      0.41      0.41       607
      weighted avg       0.44      0.43      0.43       607



### Prediccion equipo local

In [10]:
# === Definir modelo y grid de hiperparámetros ===
model = GaussianNB()
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]
}

# === GridSearchCV para encontrar el mejor var_smoothing ===
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_loc, y_train_loc)

# === Entrenamiento final con los mejores parámetros ===
best_params = grid_search.best_params_
best_model = GaussianNB(**best_params)
best_model.fit(X_train_loc, y_train_loc)

# === Predicción y evaluación ===
y_test_pred = best_model.predict(X_test_loc)

print("Mejores hiperparámetros:", best_params)

c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Mejores hiperparámetros: {'var_smoothing': 1e-09}


In [12]:

# === Evaluación del modelo ===
accuracy = accuracy_score(y_test_loc, y_test_pred)
print(f"Accuracy del modelo Naive Bayes: {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_loc, y_test_pred))

# Informe por clase
print("Informe de clasificación:")
print(classification_report(y_test_loc, y_test_pred))


Accuracy del modelo Naive Bayes: 0.2158
Matriz de confusión:
[[56 30 24 10  4  8  0  0]
 [77 28 10 34 12 32  1  0]
 [62 20 22 21 12 34  1  0]
 [14  6 13 17  4 13  1  0]
 [ 6  4  4  5  6  5  0  0]
 [ 0  1  3  0  2  2  0  0]
 [ 1  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  1  0  0]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.26      0.42      0.32       132
           1       0.31      0.14      0.20       194
           2       0.29      0.13      0.18       172
           3       0.20      0.25      0.22        68
           4       0.15      0.20      0.17        30
           5       0.02      0.25      0.04         8
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1

    accuracy                           0.22       607
   macro avg       0.15      0.17      0.14       607
weighted avg       0.27      0.22      0.22       607



c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Predicción equipo visitante

In [13]:
# === Definir modelo y grid de hiperparámetros ===
model = GaussianNB()
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7]
}

# === GridSearchCV para encontrar el mejor var_smoothing ===
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_vis, y_train_vis)

# === Entrenamiento final con los mejores parámetros ===
best_params = grid_search.best_params_
best_model = GaussianNB(**best_params)
best_model.fit(X_train_vis, y_train_vis)

# === Predicción y evaluación ===
y_test_pred = best_model.predict(X_test_vis)

print("Mejores hiperparámetros:", best_params)


c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Mejores hiperparámetros: {'var_smoothing': 1e-09}


In [14]:

# === Evaluación del modelo ===
accuracy = accuracy_score(y_test_vis, y_test_pred)
print(f"Accuracy del modelo Naive Bayes (goles visitante): {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_vis, y_test_pred))

# Informe por clase
print("Informe de clasificación:")
print(classification_report(y_test_vis, y_test_pred))


Accuracy del modelo Naive Bayes (goles visitante): 0.2422
Matriz de confusión:
[[89 28 21 13 32  6  0]
 [98 30 17 22 27 20  0]
 [49 23 13 14 23  4  0]
 [18  6  3  8 15  3  0]
 [ 5  4  4  0  6  0  0]
 [ 1  0  0  0  1  1  0]
 [ 1  0  1  1  0  0  0]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.34      0.47      0.40       189
           1       0.33      0.14      0.20       214
           2       0.22      0.10      0.14       126
           3       0.14      0.15      0.14        53
           4       0.06      0.32      0.10        19
           5       0.03      0.33      0.05         3
           6       0.00      0.00      0.00         3

    accuracy                           0.24       607
   macro avg       0.16      0.22      0.15       607
weighted avg       0.28      0.24      0.24       607



c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Miguel\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
